Video quality checking.

In [2]:
import os
import shutil
import cv2
import numpy as np

def check_video_resolution(video_path, min_width=300, min_height=300):
    """Check video resolution (Width x Height)"""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise Exception(f"Error opening video file: {video_path}")
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    
    return width >= min_width and height >= min_height

def check_video_fps(video_path, min_fps=24):
    """Check frames per second (FPS)"""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise Exception(f"Error opening video file: {video_path}")
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    return fps >= min_fps

def check_for_compression_artifacts(video_path, artifact_threshold=0.1):
    """Check for visual compression artifacts (blurring or pixelation) using simple pixel comparison"""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise Exception(f"Error opening video file: {video_path}")
    
    ret, prev_frame = cap.read()
    if not ret:
        raise Exception(f"Error reading the first frame of the video: {video_path}")
    
    artifact_detected = False
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Check every 10th frame to see if it shows signs of pixelation or blurring
    for i in range(10, frame_count, 10):
        ret, frame = cap.read()
        if not ret:
            break
        
        # Compute the difference between consecutive frames
        diff = cv2.absdiff(prev_frame, frame)
        gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
        _, threshold = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)
        
        # Check if the number of non-zero pixels (indicating compression artifacts) is too high
        non_zero_count = np.count_nonzero(threshold)
        if non_zero_count > (frame.shape[0] * frame.shape[1] * artifact_threshold):  # Percentage of pixels
            artifact_detected = True
            break
        
        prev_frame = frame
    
    cap.release()
    return artifact_detected

def video_quality_check(video_path):
    """Perform full video quality check"""
    print(f"Starting video quality check for: {video_path}")
    
    # Check resolution
    if not check_video_resolution(video_path):
        print("Resolution check failed. Video is too low resolution.")
        return False
    
    # Check FPS
    if not check_video_fps(video_path):
        print("FPS check failed. Video has too low FPS.")
        return False
    
    # Check for compression artifacts
    if check_for_compression_artifacts(video_path):
        print("Compression artifacts detected. Video quality is poor.")
        return False
    
    print("Video passed all quality checks.")
    return True

def move_video_if_quality_fails(video_path, failed_folder="../quality_failed_clips"):
    if not os.path.exists(failed_folder):
        os.makedirs(failed_folder)

    """Check video quality and remove it if it fails the checks"""
    if not video_quality_check(video_path):
        print(f"Video {video_path} failed quality checks. Moving it to {failed_folder}...")
        # Extract the video filename and construct the destination path
        video_filename = os.path.basename(video_path)
        destination_path = os.path.join(failed_folder, video_filename)
        
        # Move the video to the failed folder
        shutil.move(video_path, destination_path)
        
        print(f"Video {video_path} moved to {destination_path}.")
    else:
        print(f"Video {video_path} passed quality checks.")

def process_videos_in_folder(folder_path):
    """Loop through all video files in the given folder and check their quality"""
    # Loop through all files in the folder and check video files
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            
            # Check if the file is a video file (you can expand this list as needed)
            if file.lower().endswith(('.mp4', '.mov')):
                move_video_if_quality_fails(file_path)

if __name__ == "__main__":
    folder_path = "../dataset_clips"  # Replace with your folder path
    process_videos_in_folder(folder_path)


Starting video quality check for: ../dataset_clips\april_0.MOV
Video passed all quality checks.
Video ../dataset_clips\april_0.MOV passed quality checks.
Starting video quality check for: ../dataset_clips\april_1.MOV
Video passed all quality checks.
Video ../dataset_clips\april_1.MOV passed quality checks.
Starting video quality check for: ../dataset_clips\april_10.MOV
Video passed all quality checks.
Video ../dataset_clips\april_10.MOV passed quality checks.
Starting video quality check for: ../dataset_clips\april_11.MOV
Video passed all quality checks.
Video ../dataset_clips\april_11.MOV passed quality checks.
Starting video quality check for: ../dataset_clips\april_12.MOV
Video passed all quality checks.
Video ../dataset_clips\april_12.MOV passed quality checks.
Starting video quality check for: ../dataset_clips\april_13.MOV
Video passed all quality checks.
Video ../dataset_clips\april_13.MOV passed quality checks.
Starting video quality check for: ../dataset_clips\april_14.MOV
Vide